### Load packages


In [4]:
%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [5]:
import pandas as pd
import numpy as np

import sweetviz as sv
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots

### Load Data


In [6]:
features = pd.read_csv("data/features.csv")
stores = pd.read_csv("data/stores.csv")
sampleSubmission = pd.read_csv("data/sampleSubmission.csv")
test_df = pd.read_csv("data/test.csv")
train_df = pd.read_csv("data/train.csv")

#### Features


In [7]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [8]:
report = sv.analyze(features)
report.show_html("Report_Features.html")

                                             |          | [  0%]   00:00 -> (? left)

Report Report_Features.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


#### Features vs Train vs Test


In [9]:
print(train_df[['Store', 'Date', 'IsHoliday']].drop_duplicates().shape)
print(test_df[['Store', 'Date', 'IsHoliday']].drop_duplicates().shape)
print(features[['Store', 'Date', 'IsHoliday']].drop_duplicates().shape)

# rows of train + test = rows of features

(6435, 3)
(1755, 3)
(8190, 3)


#### Make sure one Date has one Holiday


In [10]:
assert features.groupby(['Date'])['IsHoliday'].nunique().reset_index().sort_values('IsHoliday').max().all() == 1

#### Stores


In [11]:
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [12]:
report = sv.analyze(stores)
report.show_html("Report_Stores.html")

                                             |          | [  0%]   00:00 -> (? left)

Report Report_Stores.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


##### Stores Types vs Size


In [13]:
# stores.groupby(['Type'])['Size'].agg(['mean', 'min', 'max']).reset_index()
fig = px.box(stores, x="Type", y="Size")
fig.show()

##### Dept per Store


In [14]:
print(train_df['Store'].nunique(), test_df['Store'].nunique())
display(train_df.groupby('Store').agg({'Dept':'nunique'}).reset_index().agg({'Dept':['min', 'max', 'mean']}))
display(test_df.groupby('Store').agg({'Dept':'nunique'}).reset_index().agg({'Dept':['min', 'max', 'mean']}))

45 45


,Dept
min,61.000000
max,79.000000
mean,74.022222


,Dept
min,56.000000
max,76.000000
mean,70.422222


## Merge into one df


In [15]:
final_train = train_df.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
final_train = final_train.merge(stores, on=['Store'], how='left')
final_train.head()


,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,A,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,A,151315


In [16]:
final_test = test_df.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
final_test = final_test.merge(stores, on=['Store'], how='left')
final_test.head()


,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,A,151315
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,A,151315
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,A,151315
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,A,151315
4,1,1,2012-11-30,False,52.34,3.207,2460.03,NaN,3838.35,150.57,6966.34,223.610984,6.573,A,151315


In [17]:
final_train['Date'] = pd.to_datetime(final_train['Date'], dayfirst=True)

In [18]:
train_null_perc = final_train.isnull().mean()*100
fig = px.bar(x = train_null_perc.index, y=train_null_perc.values, 
             labels={'x': 'Column', 'y': 'Percentage of Nulls'},
             title='Percentage of Null Values in Each TRAIN Column')
fig.show()

In [19]:
test_null_perc = final_test.isnull().mean()*100
fig = px.bar(x = test_null_perc.index, y=test_null_perc.values, 
             labels={'x': 'Column', 'y': 'Percentage of Nulls'},
             title='Percentage of Null Values in Each TEST Column')
fig.show()

## Add Time dimensions


In [29]:
final_train['Year'] = final_train['Date'].dt.year
final_train['Month'] = final_train['Date'].dt.month
final_train['Week'] = [f"W{w:02}"for w in final_train['Date'].dt.isocalendar().week]
final_train['Day of Week'] = final_train['Date'].dt.day_name()
final_train['Month Name'] = final_train['Date'].dt.month_name()


## Get Time Figures


### Yearly Sales


In [ ]:
smy = final_train.groupby('Year', as_index=False).agg({'Weekly_Sales':['sum', 'size']})
smy.columns = ['_'.join(i).rstrip('_') for i in smy.columns.values]
smy = smy.melt(id_vars='Year')
# fig = px.line(data_frame=smy.query("variable == 'Weekly_Sales_sum'"), x="Year", y="value")
# fig.add_bar(data_frame=smy.query("variable == 'Weekly_Sales_size'"), x="Year", y="value" )
# fig.show()

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(
    x=smy[smy['variable'] == 'Weekly_Sales_sum']['Year'],
    y=smy[smy['variable'] == 'Weekly_Sales_sum']['value'],
    name='Weekly Sales Sum'
))

fig.add_trace(go.Scatter(
    x=smy[smy['variable'] == 'Weekly_Sales_size']['Year'],
    y=smy[smy['variable'] == 'Weekly_Sales_size']['value'],
    name='Weekly Sales Size',
    yaxis='y2'
))

fig.update_layout(title='Bar and Line Chart for Weekly Sales Sum and Size',
                  yaxis=dict(title='Weekly Sales Sum'),
                  yaxis2=dict(title='Weekly Sales Size', overlaying='y', side='right', range=[0, None]))

pyo.iplot(fig)

### Weekly Sales per Year


In [33]:
smy = (
    final_train.groupby(["Year", "Week"], as_index=False)
    .agg({"Weekly_Sales": ["mean", "size"]})
    .sort_values("Week")
)
smy.columns = ["_".join(i).rstrip("_") for i in smy.columns.values]
smy = smy.melt(id_vars=["Year", "Week"])
fig = px.line(
    data_frame=smy.query("variable == 'Weekly_Sales_mean'"),
    x="Week",
    y="value",
    labels={"value": "Avg Weekly Sales"},
    color="Year",
    title="Avg Weekly Sales",
)
fig.show()

### Holidays


In [51]:
holiday = {
    "W06": "Super Bowl",
    "W36": "Labor day",
    "W47": "Thanksgiving",
    "W51": "Christmas",
}
final_train["Holiday"] = [
    holiday.get(w) if w in holiday.keys() else "Non Holiday"
    for w in final_train.Week.values
]

smy = (
    final_train[["Year", "Holiday", "Weekly_Sales"]]
    .groupby(["Year", "Holiday"], as_index=False)
    .mean()
)

fig = px.histogram(
    smy,
    x="Year",
    y="Weekly_Sales",
    color="Holiday",
    barmode="group",
    template="plotly+presentation",
    text_auto=".4s",
)
fig.update_layout(
    yaxis_title="Avg Sales",
)
fig.update_traces(textfont_size=12, textposition="outside")
fig.show()

In [39]:
holiday.get("W06")

'Super Bowl'